In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression

# Load Data

In [ ]:
files = {
    'c24': [],
    'c32': [],
    'cx24': [],
    'cx32': [],
    'ia160': [],
    'ia224': [],
    'ia288': [],
    'iax160': [],
    'iax224': [],
    'iax288': [],
}
verbose_label = {
    'c24': 'cifar100_resnet18_r24_5_601_5',
    'c32': 'cifar100_resnet18_r32_5_561_5',
    'cx24': 'cifar100_resnet18_r24_5_431_5_xla',
    'cx32': 'cifar100_resnet18_r32_5_581_5_xla',
    'ia160': 'imagenet_resnet18_r160_5_341_5_amp',
    'ia224': 'imagenet_resnet18_r224_5_171_5_amp',
    'ia288': 'imagenet_resnet18_r288_5_101_5_amp',
    'iax160': 'imagenet_resnet18_r160_5_551_5_amp_xla',
    'iax224': 'imagenet_resnet18_r224_5_161_5_amp_xla',
    'iax288': 'imagenet_resnet18_r288_5_101_5_amp_xla',
}
gpu_label = ['gpu01', 'gpu02', 'gpu03', 'gpu04']

In [ ]:
for gl in gpu_label:
    for key in files.keys():
        files[key].append(
            np.load(f'../time_record/{gl}/time_{verbose_label[key]}.npy', allow_pickle=True).item()
        )

In [ ]:
for key, value in files['c24'][0].items():
    print(key, end=', ')

# Create Measurement, Linear Regression Model, and Prediction

In [ ]:
measurement = {}

for key, value in files.items():
    measurement[key] = {
        'batch_size': np.array(value[0]['batch_size']),
        'batch_time': np.zeros_like(value[0]['batch_size'], dtype=float)
    }
    for item in value:
        measurement[key]['batch_time'] += item['avg_train_time']
    measurement[key]['batch_time'] /= len(gpu_label)

In [ ]:
reg_model = {}

for key, value in measurement.items():
    reg_model[key] = LinearRegression().fit(
        value['batch_size'].reshape(-1, 1),
        value['batch_time'],
    )
    print(key, reg_model[key].intercept_, reg_model[key].coef_)

In [ ]:
prediction = {}

for key, value in reg_model.items():
    prediction[key] = {
        'batch_size': np.arange(1, measurement[key]['batch_size'][-1] + 1),
    }
    prediction[key]['batch_time'] = value.predict(prediction[key]['batch_size'].reshape(-1, 1))
    if 'c' in key: # cifar
        prediction[key]['epoch_time'] = (
            prediction[key]['batch_time'] * np.ceil(50000 / prediction[key]['batch_size'])
        )
    elif 'i' in key: # imagenet
        prediction[key]['epoch_time'] = (
            prediction[key]['batch_time'] * np.ceil(1281167 / prediction[key]['batch_size'])
        )
    else:
        prediction[key]['epoch_time'] = None

# Analysis

In [ ]:
func = {
    'c': {
        'intercept_ls': [],
        'coef_ls': [],
        'large_batch_size_ls': [600, 560],
        'small_batch_size_ls': [],
        'total_data_amount': 50000,
    },
    'cx': {
        'intercept_ls': [],
        'coef_ls': [],
        'large_batch_size_ls': [430, 580],
        'small_batch_size_ls': [],
        'total_data_amount': 50000,
    },
    'ia': {
        'intercept_ls': [],
        'coef_ls': [],
        'large_batch_size_ls': [340, 170, 100],
        'small_batch_size_ls': [],
        'total_data_amount': 1281167,
    },
    'iax': {
        'intercept_ls': [],
        'coef_ls': [],
        'large_batch_size_ls': [550, 160, 100],
        'small_batch_size_ls': [],
        'total_data_amount': 1281167,
    },
}

for key, value in reg_model.items():
    flag = None
    if 'c' == key[0]:
        flag = 'cx' if 'x' in key else 'c'
    elif 'i' == key[0]:
        flag = 'iax' if 'x' in key else 'ia'
    else:
        raise ValueError(f'flag value "{flag}" has problem')
    func[f'{flag}']['intercept_ls'].append(reg_model[key].intercept_)
    func[f'{flag}']['coef_ls'].append(reg_model[key].coef_[0])

for key1 in func:
    print(key1, end=', ')
print()
for key2 in func['c']:
    print(key2, end=', ')

In [ ]:
extra_time_ratio = 1.05
num_total = 4
num_small = 0
num_large = num_total - num_small

for key, value in func.items():
    value['large_data_amount'] = round(
        extra_time_ratio * value['total_data_amount'] / num_total
    ) if num_small else round(value['total_data_amount'] / num_total)
    value['small_data_amount'] = round(
        (value['total_data_amount'] - value['large_data_amount'] * num_large) / num_small
    ) if num_small else 0
    temp_small_bs_ls = []
    for intercept, coef, largeBS in zip(
        value['intercept_ls'], value['coef_ls'], value['large_batch_size_ls']
    ):
        time_origin = (coef + intercept / largeBS) * value['total_data_amount'] / num_total
        time_new = time_origin * extra_time_ratio
        temp_small_bs_ls.append(
            round(intercept / (time_new / value['small_data_amount'] - coef)) if num_small
            else 0
        )
    value['small_batch_size_ls'] = temp_small_bs_ls

func

# Plot Figure

In [ ]:
DPI = 72 # [72, 150, 240, 300]

In [ ]:
# plot files
for key, value_ls in files.items():
    plt.figure(dpi=DPI)
    for gl, value in zip(gpu_label, value_ls):
        plt.plot(value['batch_size'], value['avg_train_time'], label=f'file {gl}')
    plt.plot(prediction[key]['batch_size'], prediction[key]['batch_time'], '--', label = 'prediction')
    plt.title(key)
    plt.xlabel('Batch Size')
    plt.ylabel('Training Time for a Batch (sec)')
    plt.legend()
    plt.show()

In [ ]:
plt.figure(dpi=DPI)
for key, value in measurement.items():
    if 'c' in key:
        plt.plot(value['batch_size'], value['batch_time'], label=key + ', measurement')
for key, value in prediction.items():
    if 'c' in key:
        plt.plot(value['batch_size'], value['batch_time'], '--', label=key + ', prediction')
plt.title('Training CIFAR-100 on ResNet-18')
plt.xlabel('Batch Size')
plt.ylabel('Training Time for a Batch (sec)')
plt.legend()
plt.show()

In [ ]:
plt.figure(dpi=DPI)
for key, value in measurement.items():
    if 'i' in key:
        plt.plot(value['batch_size'], value['batch_time'], label=key + ', measurement')
for key, value in prediction.items():
    if 'i' in key:
        plt.plot(value['batch_size'], value['batch_time'], '--', label=key + ', prediction')
plt.title('Training ImageNet on ResNet-18')
plt.xlabel('Batch Size')
plt.ylabel('Training Time for a Batch (sec)')
plt.legend()
plt.show()

In [ ]:
plt.figure(dpi=DPI)
for key, value in prediction.items():
    if 'c' in key:
        plt.plot(
            value['batch_size'],
            value['epoch_time'],
            #'--' if 'x' in key else '-',
            label=key + ', prediction',
        )
plt.title('Training CIFAR-100 on ResNet-18')
plt.xlabel('Batch Size')
plt.ylabel('Training Time for an Epoch (sec)')
plt.legend()
plt.show()

In [ ]:
plt.figure(dpi=DPI)
for key, value in prediction.items():
    if 'i' in key:
        plt.plot(
            value['batch_size'],
            value['epoch_time'],
            #'--' if 'x' in key else '-',
            label=key + ', prediction',
        )
plt.title('Training ImageNet on ResNet-18')
plt.xlabel('Batch Size')
plt.ylabel('Training Time for an Epoch (sec)')
plt.legend()
plt.show()